CONNECTING MEMORY WITH LLM

1. Setup LLM (Using gemini and gcp api key)
2. Connect LLM with FAISS 
3. Add question generator
4. Create chain

In [ ]:
!pip install google-generativeai langchain-google-genai python-dotenv

In [13]:
import os
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI


In [30]:
# Load .env file
load_dotenv()

True

In [ ]:
# Setting up LLM using Gemini

def load_llm():
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash-latest",
        google_api_key=os.getenv("GEMINI_API_KEY"),
        temperature=0.7,
        model_kwargs={
            "max_outputs_tokens": 6000
        }
       
    )
    
    return llm

In [44]:

# Connecting LLM with FAISS 

DB_FAISS_PATH="vectorstore/db_faiss"

custom_prompt_template= PromptTemplate(
    input_variables=["context", "question"],
    template=""""
Use the pieces and chunks of information provided as data to you to answer user's questions. 
If you dont know the answers, just say that you dont have knowledge for this and that you dont know, dont try to make up an excuse or an answer. 
Do not provide any answer out of the given and asked context.
Do not entertain obscene questions.
Be a good B.Tech tutor.
Give a long, detailed answer. Add examples and explain step by step
Use bullet points if needed.

Context: {context}
Question: {question}

Start the answer by greeting or saying hi to the user.
Give the answer from the next line.
Avoid small talk please.
""")

def set_custom_prompt(custom_prompt_template):
    prompt=PromptTemplate(template=custom_prompt_template, input_variables=["context", "question"])
    return prompt

#Loading DB
embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db=FAISS.load_local(DB_FAISS_PATH, embedding_model, allow_dangerous_deserialization=True )


In [ ]:
# Create a chain

In [45]:
# Question Generator
question_gen_prompt = PromptTemplate(
    input_variables=["chat_history", "question"],
    template="""
    Given the following conversation history and a follow-up question, rephrase the follow-up question to be a standalone question.

    Chat History:
    {chat_history}

    Follow-Up Question:
    {question}

    Standalone Question:
    """
)

llm = load_llm()

combine_docs_chain = load_qa_chain(
    llm=llm,  # =load_llm("mistralai/Mistral-7B-Instruct-v0.2"),
    chain_type="stuff",
    prompt=custom_prompt_template
)

question_generator = LLMChain(
    llm=llm,  # =load_llm("mistralai/Mistral-7B-Instruct-v0.2"),
    prompt=question_gen_prompt
)


In [46]:
llm = load_llm()
print(llm.invoke("Tell me a fact."))

content='Cleopatra lived closer in time to the invention of the iPhone than to the building of the Great Pyramid of Giza.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--b18be282-834b-431e-b1a5-27ba56275753-0' usage_metadata={'input_tokens': 5, 'output_tokens': 25, 'total_tokens': 30, 'input_token_details': {'cache_read': 0}}


In [47]:
retriever=db.as_retriever(search_kwargs={'k':3})

# Combining 

qa_chain=ConversationalRetrievalChain(
    retriever=retriever,
    combine_docs_chain=combine_docs_chain,
    question_generator=question_generator,
    return_source_documents=True   
)

In [50]:
# Invoking
user_query=input('Write your question: ')
chat_history=''
response=qa_chain.invoke({'question': user_query,
                          'chat_history': chat_history})
print("RAW RESPONSE:", response)

print("RESULT:", response.get('answer', response))
print("SOURCE DOCUMENTS:", response.get('source_documents', 'N/A'))

RAW RESPONSE: {'question': 'what is an ER diagram', 'chat_history': '', 'answer': 'Hi!\n\nAn ER diagram, or Entity-Relationship diagram, is a data model that describes the database.  It visually represents the database\'s structure by showing tables (entities), the attributes within those tables, and the relationships between the tables.  Think of it as a blueprint for your database.  It helps in understanding how different pieces of information relate to each other before you actually build the database.  For example, you might have an entity called "Customers" with attributes like CustomerID, Name, Address, and Phone Number.  Another entity might be "Orders" with attributes like OrderID, OrderDate, and CustomerID.  The ER diagram would then show a relationship between "Customers" and "Orders," indicating that one customer can have multiple orders (a one-to-many relationship).  This visual representation is crucial for designing a well-structured and efficient database.  The process o